# Machine Learning Madness 2021
**Stage 1**: Use historical data to build and train models  
**Stage 2**: Predict 2021 NCAA champtionship  
**Evaluated by**: Log loss  
**Predict**: Each matchup 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/Conferences.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyCompactResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MGameCities.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MSeasons.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MSecondaryTourneyTeams.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/Cities.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MTeamSpellings.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage2/MSeconda

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import re

# Matplotlib for visualization
from matplotlib import pyplot as plt
# display plots in the notebook
%matplotlib inline 

# Seaborn for easier visualization
import seaborn as sns
# Scikit-Learn for Modeling
import sklearn
# Pickle for saving model files
import pickle

# Import Logistic Regression
from sklearn.linear_model import LogisticRegression
# Import RandomForestClassifier and GradientBoostingClassifer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Function for splitting training and test set
from sklearn.model_selection import train_test_split
# Function for creating model pipelines
from sklearn.pipeline import make_pipeline
# For standardization
from sklearn.preprocessing import StandardScaler
# Helper for cross-validation
from sklearn.model_selection import GridSearchCV
# Classification metrics (added later)
from sklearn.metrics import roc_curve, auc

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
#read in the team data
df_teams = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MTeams.csv')

The MTeams.csv data has the team name, the ID number which is a 4 digit ID, the first season they were in D1 and the last season they were in D1. If the last season date is 2021, they are currently in D1. The dataset goes back to 1985 so that's the earliest year available.

In [4]:
df_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2022
1,1102,Air Force,1985,2022
2,1103,Akron,1985,2022
3,1104,Alabama,1985,2022
4,1105,Alabama A&M,2000,2022


In [5]:
#read in the seasons data
df_seasons = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1//MSeasons.csv')

The MSeasons.csv data has the season (year) that the tournament was played, DayZero is the 0th day of the season so that the men's championship game is on day 154, whichever region comes first alphabetically is RegionW, RegionX is the region that plays RegionW in the semifinals, in the leftover 2 regions the first alphabetically is RegionY, RegionZ plays RegionY in the semifinals.

In [6]:
df_seasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,1984-10-29 00:00:00,East,West,Midwest,Southeast
1,1986,1985-10-28 00:00:00,East,Midwest,Southeast,West
2,1987,1986-10-27 00:00:00,East,Southeast,Midwest,West
3,1988,1987-11-02 00:00:00,East,Midwest,Southeast,West
4,1989,1988-10-31 00:00:00,East,West,Midwest,Southeast


In [7]:
#read in the tournament seed data data
df_seeds = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeeds.csv')

The MNCAATourneySeeds.csv data has the season (year) the tournament was played, the team ID from MTeams.csv (df_teams), and the seed. The seed is given with the first letter (W,X,Y,Z) to specify the region and a number 01 - 16 for the seed. Play-in teams will also have an 'a' or a 'b' to distinguish which team was ranked lower ('b').

In [8]:
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [9]:
#read in the regular season results data
df_regseason = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')

The MRegularSeasonCompactResults.csv data has the season (year) of the game, DayNum is which day of the season that the game occured (this can be combined with DayZero of the MSeasons.csv (df_seasons) data to get an exact date), WTeamID is the team ID (MTeams.csv / df_teams) of the winning team, WScore is the score of the winning team, LTeamID is the team ID of the losing team, LScore is the score of the losing Team, NumOT is the number of overtimes played in that game, and WLoc is the location of the winning team (H stands for home, A stands for away, and N is a neutral site).

In [10]:
df_regseason.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [11]:
#read in the tournament results data
df_tourneyresults = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')

The MNCAATourneyCompactResults.csv data is formatted exactly the same way as the regular season results data (MRegularSeasonCompactResults.csv / df_regseason). The site is always neutral and therefore N. DayNum 134/135 are the play-in games, DayNum 136/137 are Round 1, DayNum 138/139 are Round 2, DayNum 143/144 are Round 3 (sweet sixteen), DayNum 145/146 are Round 4 (elite eight), DayNum 152 is final four, and DayNum 154 is the championship game. 

In [12]:
df_tourneyresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [13]:
#read in the regular season detailed results data
df_regseasonDets = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')

The MRegularSeasonDetailedResults.csv data has the game by game details of regular season games since the 2002-2003 season. The first 8 columns are the same as in the compact results file (MRegularSeasonCompactResults.csv / df_regseason). WFGM is the # of field goals made by the winning team, WFGA is the # of field goals attempted by the winning team. WFGM3 is the 3 pointers made by the winning team, WFGA3 is the # of 3 pointers attempted. WFTM is the # of free throws made by the winning team, WFTA is the # of free throws attempted by the winning team. WOR is the # of offensive rebounds pulled by the winning team, WDR is the # of defensive rebounds pulled by the winning team. WAst is the # of assists by the winning team. WTO is the # of turnovers committed by the winning team. WStl is the # of steals by the winning team, WBlk is the # of blocks by the winning team. WPF is the # of personal fouls committed by the winning team. Then there are the same stats but with L instead of W for the losing team.

In [14]:
df_regseasonDets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [15]:
#read in the tournament detailed results data
df_tourneyDets = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')

MNCAATourneyDetailedResults.csv has the same information as the detailed regular seadon data but for the tournament.

In [16]:
df_tourneyDets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [17]:
#read in the cities data
df_cities = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/Cities.csv')

cities.csv has the data for the cities where the games were held. CityID is a unique 4 digit number for the city, City is the text name of the city and State is the state the citie is in.

In [18]:
df_cities.head()

,CityID,City,State
0,4001,Abilene,TX
1,4002,Akron,OH
2,4003,Albany,NY
3,4004,Albuquerque,NM
4,4005,Allentown,PA


In [19]:
#read in the games city data
df_gamecities = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MGameCities.csv')

MGameCities.csv has all the game cities data since 2010. Season, DayNum, WTeamID, LTeamID, and CityID have been discussed before. CRType is either Regular for a regular season game, NCAA is a tournament game, and Secondary is a game from the secondary data.

In [20]:
df_gamecities.head()

,Season,DayNum,WTeamID,LTeamID,CRType,CityID
0,2010,7,1143,1293,Regular,4027
1,2010,7,1314,1198,Regular,4061
2,2010,7,1326,1108,Regular,4080
3,2010,7,1393,1107,Regular,4340
4,2010,9,1143,1178,Regular,4027


In [21]:
#read in the rankings data
df_rankings = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')

The MMasseyOrdinals.csv has ranking information from as far back as 2002. Season and TeamID have already been discussed. RankingDayNum has the day number that it is appropriate to use the ranking information for predicting games. For example, you shouldn't use a ranking on day 110 to predict a game on day 100. SystemName is a 3 letter abbreviation for the ranking system. OrdinalRank is the ranking of the teams on that day in that system. By convention the final ranking before the tournament is RankingDayNum = 133.

In [22]:
df_rankings.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [23]:
#read in the tournament slot data
df_slots = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySlots.csv')

MNCAATourneySlots.csv data can be used to identify what round a historical tournament game occured in and what seed/slot the teams that played that game were. This could be useful for simulating the progression of a bracket. Slot uniquely identifies one of the tournament games. For play-in games, it is a three-character string identifying the seed fulfilled by the winning team, such as W16 or Z13. For regular tournament games, it is a four-character string, where the first two characters tell you which round the game is (R1, R2, R3, R4, R5, or R6) and the second two characters tell you the expected seed of the favored team. For example, the R2W1 slot indicates the Round 2 game that would have the 1 seed from the W bracket, assuming that all favored teams have won up to that point. Even if that R2W1 slot were actually a game between the W09 and W16 teams, it is still considered to be the R2W1 slot. The slot names are different for the final two rounds, where R5WX identifies the national semifinal game between the winners of regions W and X, and R5YZ identifies the national semifinal game between the winners of regions Y and Z, and R6CH identifies the championship game.

StrongSeed indicates the expected stronger-seeded team that plays in this game. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the MNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column. In the first record of this file (slot R1W1), we see that seed W01 is the "StrongSeed", which during the 1985 tournament would have been Georgetown. Whereas for games from Round 2 or later, rather than a team seed, we will see a "slot" referenced in this column. So in the 33rd record of this file (slot R2W1), it tells us that the winners of slots R1W1 and R1W8 will face each other in Round 2. Of course, in the last few games of the tournament - the national semifinals and finals - it's not really meaningful to talk about a "strong seed" or "weak seed", since you would have #1 seeds favored to face each other, but those games are nevertheless represented in the same format for the sake of consistency. WeakSeed indicates the expected weaker-seeded team that plays in this game, assuming all favored teams have won so far. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the MNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column.

In [24]:
df_slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [25]:
#read in the tournament slot data
df_seed = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeedRoundSlots.csv')

MNCAATourneySeedRoundSlots.csv represents the bracket structure. Seed is the tournament seed of the team. GameRound is the round during the tournament that the game would occur in, where Round 0 (zero) is for the play-in games, Rounds 1/2 are for the first weekend, Rounds 3/4 are for the second weekend, and Rounds 5/6 are the national semifinals and finals. GameSlot is the game slot that the team would be playing in, during the given GameRound. The naming convention for slots is the same as in df_slots. EarlyDayNum and LateDayNum describe the earliest possible, and latest possible, DayNums that the game might be played on.

In [26]:
df_seed.head()

,Seed,GameRound,GameSlot,EarlyDayNum,LateDayNum
0,W01,1,R1W1,136,137
1,W01,2,R2W1,138,139
2,W01,3,R3W1,143,144
3,W01,4,R4W1,145,146
4,W01,5,R5WX,152,152


There is a lot of data that I don't plan to use here. I want to take the regular season data of wins and losses, as well as the detailed regular season information, and the tournament seeds to predict the tournament results. So now I need to make that dataframe with just that information.

In [27]:
df = pd.DataFrame()

The first thing I want to do is a little feature engineering. The detailed regular season data has stats for each game which is great. I'm going to want to turn these per game stats into averages per season. So I'll take the TeamID and compute the averages for that team. 

In [28]:
df_regseasonDets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


Let's start by turning all the attempts and made shots into a percentage.

In [29]:
df_regseasonDets['WFGP'] = df_regseasonDets.WFGM / df_regseasonDets.WFGA
df_regseasonDets['WFGP3'] = df_regseasonDets.WFGM3 / df_regseasonDets.WFGA3
df_regseasonDets['WFTP'] = df_regseasonDets.WFTM / df_regseasonDets.WFTA
df_regseasonDets['LFGP'] = df_regseasonDets.LFGM / df_regseasonDets.LFGA
df_regseasonDets['LFGP3'] = df_regseasonDets.LFGM3 / df_regseasonDets.LFGA3
df_regseasonDets['LFTP'] = df_regseasonDets.LFTM / df_regseasonDets.LFTA

In [30]:
df_regseasonDets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LTO,LStl,LBlk,LPF,WFGP,WFGP3,WFTP,LFGP,LFGP3,LFTP
0,2003,10,1104,68,1328,62,N,0,27,58,...,18,9,2,20,0.465517,0.214286,0.611111,0.415094,0.200000,0.727273
1,2003,10,1272,70,1393,63,N,0,26,62,...,12,8,6,16,0.419355,0.400000,0.526316,0.358209,0.250000,0.450000
2,2003,11,1266,73,1437,61,N,0,24,58,...,12,2,5,23,0.413793,0.444444,0.586207,0.301370,0.115385,0.608696
3,2003,11,1296,56,1457,50,N,0,18,38,...,19,4,3,23,0.473684,0.333333,0.548387,0.367347,0.272727,0.533333
4,2003,11,1400,77,1208,71,N,0,30,61,...,10,7,1,14,0.491803,0.428571,0.846154,0.387097,0.375000,0.629630


Let's create a dataframe for that has the average information of the games grouped by season and TeamID. I'm going to do this by first making two different dataframes, one for the winning team stats, and one for the losing. Then I'll concatenate them together and then take the averages. 

In [31]:
df_WregseasonDets = pd.DataFrame() 
df_WregseasonDets['TeamID'] = df_regseasonDets.WTeamID
df_WregseasonDets['Season'] = df_regseasonDets.Season
df_WregseasonDets['Score'] = df_regseasonDets.WScore
df_WregseasonDets['FGP'] = df_regseasonDets.WFGP
df_WregseasonDets['FGP3'] = df_regseasonDets.WFGP3
df_WregseasonDets['FTP'] = df_regseasonDets.WFTP
df_WregseasonDets['OR'] = df_regseasonDets.WOR
df_WregseasonDets['DR'] = df_regseasonDets.WDR
df_WregseasonDets['Ast'] = df_regseasonDets.WAst
df_WregseasonDets['TO'] = df_regseasonDets.WTO
df_WregseasonDets['Blk'] = df_regseasonDets.WBlk
df_WregseasonDets['Stl'] = df_regseasonDets.WStl
df_WregseasonDets['PF'] = df_regseasonDets.WPF

In [32]:
df_WregseasonDets.head()

,TeamID,Season,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Blk,Stl,PF
0,1104,2003,68,0.465517,0.214286,0.611111,14,24,13,23,1,7,22
1,1272,2003,70,0.419355,0.400000,0.526316,15,28,16,13,4,4,18
2,1266,2003,73,0.413793,0.444444,0.586207,17,26,15,10,2,5,25
3,1296,2003,56,0.473684,0.333333,0.548387,6,19,11,12,2,14,18
4,1400,2003,77,0.491803,0.428571,0.846154,17,22,12,14,4,4,20


In [33]:
df_WregseasonDets.shape

(100423, 13)

In [34]:
df_LregseasonDets = pd.DataFrame() 
df_LregseasonDets['TeamID'] = df_regseasonDets.LTeamID
df_LregseasonDets['Season'] = df_regseasonDets.Season
df_LregseasonDets['Score'] = df_regseasonDets.LScore
df_LregseasonDets['FGP'] = df_regseasonDets.LFGP
df_LregseasonDets['FGP3'] = df_regseasonDets.LFGP3
df_LregseasonDets['FTP'] = df_regseasonDets.LFTP
df_LregseasonDets['OR'] = df_regseasonDets.LOR
df_LregseasonDets['DR'] = df_regseasonDets.LDR
df_LregseasonDets['Ast'] = df_regseasonDets.LAst
df_LregseasonDets['TO'] = df_regseasonDets.LTO
df_LregseasonDets['Blk'] = df_regseasonDets.LBlk
df_LregseasonDets['Stl'] = df_regseasonDets.LStl
df_LregseasonDets['PF'] = df_regseasonDets.LPF

In [35]:
df_LregseasonDets.head()

,TeamID,Season,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Blk,Stl,PF
0,1328,2003,62,0.415094,0.200000,0.727273,10,22,8,18,2,9,20
1,1393,2003,63,0.358209,0.250000,0.450000,20,25,7,12,6,8,16
2,1437,2003,61,0.301370,0.115385,0.608696,31,22,9,12,5,2,23
3,1457,2003,50,0.367347,0.272727,0.533333,17,20,9,19,3,4,23
4,1208,2003,71,0.387097,0.375000,0.629630,21,15,12,10,1,7,14


In [36]:
df_LregseasonDets.shape

(100423, 13)

Concatenate df_WregseasonDets and df_LregseasonDets together.

In [37]:
df_regseasonDets_teams = pd.DataFrame()
df_regseasonDets_teams = pd.concat([df_WregseasonDets, df_LregseasonDets])

In [38]:
df_regseasonDets_teams.shape

(200846, 13)

In [39]:
df_regseasonDets_teams.head()

,TeamID,Season,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Blk,Stl,PF
0,1104,2003,68,0.465517,0.214286,0.611111,14,24,13,23,1,7,22
1,1272,2003,70,0.419355,0.400000,0.526316,15,28,16,13,4,4,18
2,1266,2003,73,0.413793,0.444444,0.586207,17,26,15,10,2,5,25
3,1296,2003,56,0.473684,0.333333,0.548387,6,19,11,12,2,14,18
4,1400,2003,77,0.491803,0.428571,0.846154,17,22,12,14,4,4,20


Take the averages by team and season.

In [40]:
df = df_regseasonDets_teams.groupby(['Season','TeamID']).agg({'Score':'mean','FGP':'mean',
                                                       'FGP3':'mean','FTP':'mean',
                                                       'OR':'mean','DR':'mean','Ast':'mean',
                                                      'TO':'mean','Stl':'mean','Blk':'mean',
                                                      'PF':'mean'})
df.head()

Score       FGP      FGP3       FTP         OR         DR  \
Season TeamID                                                                  
2003   1102    57.250000  0.486149  0.367637  0.642402   4.178571  16.821429   
       1103    78.777778  0.487294  0.331990  0.735271   9.777778  19.925926   
       1104    69.285714  0.419676  0.325442  0.705168  13.571429  23.928571   
       1105    71.769231  0.396204  0.359630  0.709598  13.500000  23.115385   
       1106    63.607143  0.425530  0.350196  0.623158  12.285714  23.857143   

                     Ast         TO       Stl       Blk         PF  
Season TeamID                                                       
2003   1102    13.000000  11.428571  5.964286  1.785714  18.750000  
       1103    15.222222  12.629630  7.259259  2.333333  19.851852  
       1104    12.107143  13.285714  6.607143  3.785714  18.035714  
       1105    14.538462  18.653846  9.307692  2.076923  20.230769  
       1106    11.678571  17.035714  8.357143  3.142857  18.178571

In [41]:
df.shape

(6892, 11)

I would also like to add the rankings before the tournament. This is day 133.

In [42]:
df_rankings.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [43]:
last_day = df_rankings['RankingDayNum']==133

In [44]:
df_rankings_last = df_rankings[last_day]

In [45]:
df_rankings_last.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
121021,2003,133,AP,1112,2
121022,2003,133,AP,1163,23
121023,2003,133,AP,1166,15
121024,2003,133,AP,1173,16
121025,2003,133,AP,1181,7


In [46]:
df_rankings_last.SystemName.unique()

array(['AP', 'ARG', 'BIH', 'BOB', 'BRZ', 'COL', 'DOL', 'DUN', 'DWH',
       'ECK', 'ENT', 'ERD', 'GRN', 'GRS', 'HER', 'HOL', 'IMS', 'MAS',
       'MKV', 'MOR', 'POM', 'RPI', 'RTH', 'SAG', 'SAU', 'SE', 'SEL',
       'STR', 'TSR', 'USA', 'WLK', 'WOB', 'WOL', 'WTE', 'BD', 'CNG',
       'DES', 'JON', 'LYN', 'MGY', 'NOR', 'REI', 'RM', 'SIM', 'ACU',
       'BCM', 'CMV', 'DC', 'KLK', 'REN', 'RIS', 'ROH', 'SAP', 'SCR',
       'WIL', 'DOK', 'JCI', 'KPK', 'MB', 'PH', 'PIG', 'PKL', 'TRX', 'CPR',
       'ISR', 'KRA', 'LYD', 'RTR', 'UCS', 'BKM', 'CPA', 'JEN', 'PGH',
       'REW', 'RSE', 'SPW', 'STH', 'BPI', 'DC2', 'DCI', 'HKB', 'LMC',
       'NOL', 'OMY', 'RTB', 'KEL', 'KMV', 'RT', 'TW', 'AUS', 'KOS', 'PEQ',
       'PTS', 'ROG', 'RTP', 'TMR', '7OT', 'ADE', 'BBT', 'BNM', 'BUR',
       'CJB', 'CRO', 'EBP', 'HAT', 'MSX', 'SFX', 'TBD', 'BLS', 'D1A',
       'DII', 'KBM', 'TPR', 'MvG', 'PPR', 'SP', 'SPR', 'STF', 'STS',
       'TRP', 'UPS', 'WMR', 'BWE', 'LOG', 'TRK', 'DAV', 'FAS', 'FSH',
       'HAS', 'H

Ok there are a lot of ranking systems here and I definitely do not want all of them. So for each team and year let's take the mean. This might help to give us an idea of how each team is ranked. I might end up just taking the seed and not caring about the rank if I don't think this is helpful information. 

In [47]:
df_rankings_last_Ave  = df_rankings_last.groupby(['Season','TeamID']).agg({'OrdinalRank':['mean']})#,'RankMin':'min','RankMax':'max'})
df_rankings_last_Ave.head()

OrdinalRank
                     mean
Season TeamID            
2003   1102     156.03125
       1103     168.00000
       1104      38.03125
       1105     308.96875
       1106     262.68750

In [48]:
df_rankings_last_Ave.shape

(6180, 1)

In [49]:
frames = [df,df_rankings_last_Ave]
df = pd.concat(frames, axis=1)

In [50]:
df.head()

Score       FGP      FGP3       FTP         OR         DR  \
Season TeamID                                                                  
2003   1102    57.250000  0.486149  0.367637  0.642402   4.178571  16.821429   
       1103    78.777778  0.487294  0.331990  0.735271   9.777778  19.925926   
       1104    69.285714  0.419676  0.325442  0.705168  13.571429  23.928571   
       1105    71.769231  0.396204  0.359630  0.709598  13.500000  23.115385   
       1106    63.607143  0.425530  0.350196  0.623158  12.285714  23.857143   

                     Ast         TO       Stl       Blk         PF  \
Season TeamID                                                        
2003   1102    13.000000  11.428571  5.964286  1.785714  18.750000   
       1103    15.222222  12.629630  7.259259  2.333333  19.851852   
       1104    12.107143  13.285714  6.607143  3.785714  18.035714   
       1105    14.538462  18.653846  9.307692  2.076923  20.230769   
       1106    11.678571  17.035714  8.357143  3.142857  18.178571   

               (OrdinalRank, mean)  
Season TeamID                       
2003   1102              156.03125  
       1103              168.00000  
       1104               38.03125  
       1105              308.96875  
       1106              262.68750

I don't love this ranking column name so I'll just rename it rank. Remember this is the mean rank.

In [51]:
list(df.columns.values)

['Score',
 'FGP',
 'FGP3',
 'FTP',
 'OR',
 'DR',
 'Ast',
 'TO',
 'Stl',
 'Blk',
 'PF',
 ('OrdinalRank', 'mean')]

In [52]:
df.rename(columns = {('OrdinalRank', 'mean'):'Rank'}, inplace = True) 

In [53]:
df.head()

Score       FGP      FGP3       FTP         OR         DR  \
Season TeamID                                                                  
2003   1102    57.250000  0.486149  0.367637  0.642402   4.178571  16.821429   
       1103    78.777778  0.487294  0.331990  0.735271   9.777778  19.925926   
       1104    69.285714  0.419676  0.325442  0.705168  13.571429  23.928571   
       1105    71.769231  0.396204  0.359630  0.709598  13.500000  23.115385   
       1106    63.607143  0.425530  0.350196  0.623158  12.285714  23.857143   

                     Ast         TO       Stl       Blk         PF       Rank  
Season TeamID                                                                  
2003   1102    13.000000  11.428571  5.964286  1.785714  18.750000  156.03125  
       1103    15.222222  12.629630  7.259259  2.333333  19.851852  168.00000  
       1104    12.107143  13.285714  6.607143  3.785714  18.035714   38.03125  
       1105    14.538462  18.653846  9.307692  2.076923  20.230769  308.96875  
       1106    11.678571  17.035714  8.357143  3.142857  18.178571  262.68750

Right now Season and TeamID are indexes and it's kicking my ass so I'm going to make them columns. 

In [54]:
df.reset_index(inplace=True)

In [55]:
df.head()

,Season,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank
0,2003,1102,57.250000,0.486149,0.367637,0.642402,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000,156.03125
1,2003,1103,78.777778,0.487294,0.331990,0.735271,9.777778,19.925926,15.222222,12.629630,7.259259,2.333333,19.851852,168.00000
2,2003,1104,69.285714,0.419676,0.325442,0.705168,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,18.035714,38.03125
3,2003,1105,71.769231,0.396204,0.359630,0.709598,13.500000,23.115385,14.538462,18.653846,9.307692,2.076923,20.230769,308.96875
4,2003,1106,63.607143,0.425530,0.350196,0.623158,12.285714,23.857143,11.678571,17.035714,8.357143,3.142857,18.178571,262.68750


Now let's add the seed to our data. Since we only have regular season data from season 2002, we're going to only use seed data from then as well.

In [56]:
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [57]:
seeds_mask = df_seeds.Season>2002

In [58]:
df_seeds2002 = df_seeds[seeds_mask]

In [59]:
df = df.merge(df_seeds2002, how = 'inner', on = ['Season', 'TeamID'])

In [60]:
df.head()

,Season,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2003,1104,69.285714,0.419676,0.325442,0.705168,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,18.035714,38.031250,Y10
1,2003,1112,85.214286,0.463563,0.351060,0.701154,15.178571,27.642857,17.642857,14.785714,8.464286,4.214286,17.750000,2.676471,Z01
2,2003,1113,75.965517,0.481680,0.328376,0.675667,13.689655,23.310345,15.551724,14.000000,5.206897,4.241379,19.413793,36.000000,Z10
3,2003,1120,70.100000,0.470055,0.355662,0.652663,11.933333,22.566667,13.400000,15.900000,8.033333,4.966667,16.366667,43.937500,W10
4,2003,1122,69.703704,0.456810,0.353542,0.682295,10.629630,24.222222,13.407407,15.222222,7.037037,4.666667,18.666667,94.968750,W13


Now that the seed is there, let's turn it into an integer by removing the region. I don't think we'll need the region data since we have the team IDs but if we do, I can always add it back in.

In [61]:
df['Seed'] = df['Seed'].str[1:]

In [62]:
df.head()

,Season,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2003,1104,69.285714,0.419676,0.325442,0.705168,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,18.035714,38.031250,10
1,2003,1112,85.214286,0.463563,0.351060,0.701154,15.178571,27.642857,17.642857,14.785714,8.464286,4.214286,17.750000,2.676471,01
2,2003,1113,75.965517,0.481680,0.328376,0.675667,13.689655,23.310345,15.551724,14.000000,5.206897,4.241379,19.413793,36.000000,10
3,2003,1120,70.100000,0.470055,0.355662,0.652663,11.933333,22.566667,13.400000,15.900000,8.033333,4.966667,16.366667,43.937500,10
4,2003,1122,69.703704,0.456810,0.353542,0.682295,10.629630,24.222222,13.407407,15.222222,7.037037,4.666667,18.666667,94.968750,13


Since we just stripped a letter from our Seed, the seed is probably still a string. So let's verify that and change it to a float. We have all of those pesky play in games as well. So for our purpose I'm changing 11a to 11.1 and 11b to 11.2 so they'll be numerical. I'm not meaning to imply that one team is ranked higher than another. If we need to change this in the future we can.

In [63]:
df.Seed.replace(['11a','11b','12a','12b','13a','13b','14a','14b','16a','16b'],
['11','11','12','12','13','13','14','14','16','16'],inplace=True)

In [64]:
df['Seed'] = df['Seed'].astype(float)

In [65]:
df.dtypes

Season      int64
TeamID      int64
Score     float64
FGP       float64
FGP3      float64
FTP       float64
OR        float64
DR        float64
Ast       float64
TO        float64
Stl       float64
Blk       float64
PF        float64
Rank      float64
Seed      float64
dtype: object

Perfect. I'm happy with this and I'll save the data as a csv file in the processed data folder.

In [66]:
df.to_csv('/kaggle/working/data_averages.csv',index=False)